# [ICTCOG AI Academy] 7기 고급시각저녁반
# 	다양한 CNN 모델 (1)

CNN_advanced.pdf 참고
### convolution

- 각각의 특징을 탐지하는 convolution 필터 여러개가 존재
  - 예. 고양이의 눈,귀,코
  - 유사할수록 값이 값이 최대화됨 ( element-wise )

- 아미지의 다양한 경우를 고려
- 위치에 덜 민감함
  - > NN은 1차원화하기 때문에 위치에 민감함
- 학습을 통해서 찾은 convolution 사람은 이해 불가
  - black box model
-  레이어가 깊어질수록 추상적인 특성을 추출

In [1]:
import tensorflow as tf

In [3]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data()

In [4]:
X_train.shape

(60000, 28, 28)

In [11]:
X_train_3d = X_train.reshape(-1,28,28,1) #dataset 3차원화

In [25]:
#가장 간단한 뼈대
inputs= tf.keras.layers.Input( shape=(28,28,1)) #channel last
x= tf.keras.layers.Conv2D( 32,3 )(inputs)       #3x3 32개 
x=tf.keras.layers.Flatten()(x)
outputs=tf.keras.layers.Dense(10)(x)

In [26]:
model=tf.keras.models.Model(inputs=inputs, outputs=outputs )
model.summary() #학습파라미터 216,650

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
flatten_6 (Flatten)          (None, 21632)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                216330    
Total params: 216,650
Trainable params: 216,650
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 컨볼루션 더 깊게 쌓기
inputs= tf.keras.layers.Input( shape=(28,28,1)) #channel last
x= tf.keras.layers.Conv2D( 32,3 )(inputs)       #3x3 32개 
x=tf.keras.layers.Activation('relu')(x)
x= tf.keras.layers.Conv2D( 64,3 )(x)       #3x3 64개 
x=tf.keras.layers.Activation('relu')(x)
x=tf.keras.layers.Flatten()(x)
outputs=tf.keras.layers.Dense(10)(x)

model=tf.keras.models.Model(inputs=inputs, outputs=outputs )

In [24]:
model.summary() #학습파라미터 387,466

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_6 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
activation_7 (Activation)    (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3686

학습파라미터가 클수록 데이터가 많이 필요함

### Pooling
- pooling으로 학습파라미터 크기를 줄임
- 이미지가 작은 경우 특징을 잘 추출하기 힘듦

In [29]:
# 컨볼루션 더 깊게 쌓기
inputs= tf.keras.layers.Input( shape=(28,28,1))
x= tf.keras.layers.Conv2D( 32,3 )(inputs)       
x=tf.keras.layers.Activation('relu')(x)
x=tf.keras.layers.MaxPool2D(2)(x)             # pooling
x= tf.keras.layers.Conv2D( 64,3 )(x)       
x=tf.keras.layers.Activation('relu')(x)
x=tf.keras.layers.MaxPool2D(2)(x)
x= tf.keras.layers.Conv2D( 64,3 )(x)        
x=tf.keras.layers.Activation('relu')(x)

x=tf.keras.layers.Flatten()(x)
outputs=tf.keras.layers.Dense(10)(x)

model=tf.keras.models.Model(inputs=inputs, outputs=outputs )
model.summary()                                # 학습 파라미터 급격히 줄어듦

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
activation_12 (Activation)   (None, 26, 26, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
activation_13 (Activation)   (None, 11, 11, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0   

#### FCNN
-  로컬 특징 잃음
- computational power $$$

#### LCNN
- locally connected
- FCNN보다 학습파라미터가 적음

#### CNN
- shared weights
- **translational invariance**
>  Local Contrast Normalization
  - LCN 효과가 없음을 실험을 통해 밝혀냄

### CNN 가정
- stationarity of statistics 통계적 안정성
- locality of pixel dependencies 
-  예측모델의 품질 측정 방법
    - statistical efficiency상대적으로 적은 데이터로 학습시켜도 좋은 성능을 내야함 
  - 필터 (H_f,W_f, C,_i, C_o) 

### CNN 문제점
- 위치에 상관없이 특징을 파악하는게 단점이 되기도함
- 캡슐 네트워크 by Hinton
  - CNN 문제점 보완

### Pooling
- pooling해서 하나의 대표값으로 바꾸어 특징 파악 가능
- 정확한 값 만 고려
- 모델의 경우를 유연성 추가
  - 약간의 회전에도 잘 파악하는 이유
- 하나의 대표값으로 바꿈

#### padding
-  크기를 동일하게 유지하고 싶은 경우 
- 가변 값들은 특징 파악이 잘 안되는 경우
- zero-padding  일반적

> No Free Lunch


### 논문
### Gradient-Based Learning Applied to Document Recognition by Yann LeCun, Yoshua Bengio
#### LeNet5
- 최초의 모델 소개
- feature maps
  - 컨볼루션한 결과
- subsampling
  - 대표값으로 줄임
- maxpooling 
- computing power의 한계
- 이미지가 작은 경우 kernel/filter 갯수 많음



In [31]:
model.layers

In [32]:
model.layers[1].weights

[<tf.Variable 'conv2d_15/kernel:0' shape=(3, 3, 1, 32) dtype=float32, numpy=
 array([[[[-0.10464747,  0.09557877,  0.11799447, -0.10255449,
            0.1392379 ,  0.01726529,  0.10698991, -0.04620139,
           -0.03272938, -0.06362067, -0.10697258, -0.1014405 ,
            0.11675908,  0.09906514,  0.02982171, -0.06852186,
           -0.14018679,  0.09533614, -0.06171113,  0.01816937,
           -0.01694323, -0.11196334, -0.01497936, -0.04581403,
            0.13030244, -0.098335  ,  0.00497927, -0.03364054,
           -0.03437515, -0.10441165, -0.11840166, -0.12051989]],
 
         [[-0.01083001,  0.0347655 , -0.1372103 , -0.08354002,
            0.02445978, -0.09108914, -0.11538833,  0.07897568,
           -0.08985642, -0.05794017,  0.12519334,  0.10769187,
            0.07565434,  0.12376811,  0.00316538,  0.11313693,
           -0.03250701,  0.12885307, -0.03211127,  0.04137519,
            0.00455365, -0.09672806, -0.08972979,  0.07300831,
            0.12717514, -0.0681629 , 

In [38]:
model.layers[1].kernel

<tf.Variable 'conv2d_15/kernel:0' shape=(3, 3, 1, 32) dtype=float32, numpy=
array([[[[-0.10464747,  0.09557877,  0.11799447, -0.10255449,
           0.1392379 ,  0.01726529,  0.10698991, -0.04620139,
          -0.03272938, -0.06362067, -0.10697258, -0.1014405 ,
           0.11675908,  0.09906514,  0.02982171, -0.06852186,
          -0.14018679,  0.09533614, -0.06171113,  0.01816937,
          -0.01694323, -0.11196334, -0.01497936, -0.04581403,
           0.13030244, -0.098335  ,  0.00497927, -0.03364054,
          -0.03437515, -0.10441165, -0.11840166, -0.12051989]],

        [[-0.01083001,  0.0347655 , -0.1372103 , -0.08354002,
           0.02445978, -0.09108914, -0.11538833,  0.07897568,
          -0.08985642, -0.05794017,  0.12519334,  0.10769187,
           0.07565434,  0.12376811,  0.00316538,  0.11313693,
          -0.03250701,  0.12885307, -0.03211127,  0.04137519,
           0.00455365, -0.09672806, -0.08972979,  0.07300831,
           0.12717514, -0.0681629 ,  0.03698729, -0.1

In [34]:
model.layers[1].bias

<tf.Variable 'conv2d_15/bias:0' shape=(32,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

In [ ]:
#초기화된 weights는 학습을 통해서 바뀌게 됨
tf.keras.layers.Conv2D(kernel_initializer=)

### Invariance vs equivariance

##### translational invariance
- input **위치에 상관없이 output이 동일한 값**을 갖는것
- CNN의 특성 
>  크기, 회전, illumination Invariance 등이 있음
- > Convolution 자체만으로 예측모델 사용 불가(segmentation에서 다시 다룰 예정)
  - data augmentation 하는 이유

> Convnet 은 convolution + FCNN => invariant 
- 이미지 크기 중요함 

##### equivariance
  - 변환하고 어떤값을 넣는 경우
  - 위치에 따라 결과값 상이함
> 학습된 이후에는 classifier 일반 ML 모델로도 가능

#### Pooling
- 계산 복잡도 줄임
- 값을 대표하면서 어느정도까지** scale, rotation 까지 invariant**하게 됨
- 유연성 증가
- pooling 사용하지 않는 이유
  - Hinton은 Pooling 통해서 복잡도를 줄일 필요 없다함
  - 논문 Striving for Simplicity : The All Convolutional Net
  - VAEs, GANs 에서 pooling 사용하지 않을때 더 좋은 성능이 나옴
  - >  No Free Lunch : 상황에 따라 다르다.

### Data Augmentaion 
- 기존의 데이터를 이용해서 새로운 데이터를 생성하면 overfitting을 방지하나 성능은 어느정도만 향상
-  feature cross (특성 교차)
- [CNN Explainer](https://poloclub.github.io/cnn-explainer/) 
  - CNN 시각적으로 이해 도움
  - 개념 익히기 좋음
[Deep Larning 책 ](https://www.deeplearningbook.org/)

- Partial Convolutional based Padding
   - Padding 에 따른 성능 비교 



In [39]:
import numpy as np
import scipy

In [ ]:
np.convolve()          # mode='full','same','valid'
scipy.ndimage.convolve # 여러가지 모드 지원

In [41]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

#MinMaxScaler
train_images, test_images = train_images / 255.0, test_images / 255.0

### scaler
- 성능 향상
  - 1 ~  0 양의 값만 갖음 
  - 1 ~ -1 양,음 둘다 나옴
    - 복잡하고, underfitting인 경우


In [44]:
model = tf.keras.models.Sequential()
#1. Feature Extractor 
# 3x3 필터 32개
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu')) #크기를 늘림
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

In [45]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [47]:
#2. Classifier 연결(위보다 간단함)
model.add(tf.keras.layers.Flatten())  #Flatten 한 후 연결
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [48]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_10 (Flatten)         (None, 576)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)               

> 논문 보는법
  1. Abstract 확인
  2. Experiments 확인
    - 실험 내용,결과 